<h3>Domain-Specific RAG Chatbot with Llama 3.1 (70B) via Groq Cloud</h3>
<p>
A Retrieval-Augmented Generation (RAG) chatbot leveraging the Llama 3.1 (70B) versatile model, accessed through Groq Cloud.</p>
<h3>To checkout the available LLM model on Groq cloud <a href="https://console.groq.com/playground   
"> clic me</a></h3>


In [27]:
from langchain_openai import  OpenAI 
from langchain_community.document_loaders.csv_loader import  CSVLoader 
from langchain_community.document_loaders import  TextLoader
from langchain.text_splitter import  RecursiveCharacterTextSplitter
from langchain_openai import  OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.vectorstores import  VectorStoreRetriever 
from langchain.chains import RetrievalQA 
from langchain.prompts import PromptTemplate
import pandas as pd 
import numpy as np 
import os 
from dotenv import  load_dotenv
from langchain_groq import ChatGroq 
load_dotenv()

True

In [29]:
groq_api_key= os.getenv('GROQ_CLOUD_API')

In [30]:
model_name = "llama-3.1-70b-versatile"
llm = ChatGroq(temperature=0.7,
                groq_api_key=groq_api_key,
                model_name=model_name)

In [6]:
response = llm.invoke("The first person land on moon was?")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. His famous words upon stepping onto the moon were: "That's one small step for man, one giant leap for mankind."


In [7]:
# File paths for the CSV files
qa_file_path = r"data\questin_answering.csv"
upflairs_qa_file_path = r"data\upflairs_question_answer.csv"

# Reading the CSV files into DataFrames
df1 = pd.read_csv(qa_file_path)
df2 = pd.read_csv(upflairs_qa_file_path)

combine_dataset = pd.concat([df2,df1])
print("No of question and answers in the dataset : ",combine_dataset.shape[0])

final_data_file_path = r"data\final_data.csv"
combine_dataset.to_csv(final_data_file_path,index=False)
print("successfully save your final dataset at : ",final_data_file_path)

No of question and answers in the dataset :  39
successfully save your final dataset at :  data\final_data.csv


In [8]:
loader = CSVLoader(file_path=final_data_file_path, source_column="Question")
data = loader.load()


In [10]:
os.environ['OPENAI_API_KEY'] = os.getenv("gpt_api_key")

In [11]:
vectordb_file_path = "fais_index"
embeding = OpenAIEmbeddings()
vectordb = FAISS.from_documents(data,embeding)

# Save vector database locally
vectordb.save_local(vectordb_file_path)

In [ ]:
query_answer = vectordb.similarity_search("tell me about upflairs?")
page_content = query_answer[0].page_content

# Split the content at "Answer:" and get the part after it
# answer = page_content.split("Answer:")[1].strip()

print(page_content)


Question: tell me about upflairs?
Answer: UpFlairs is an innovative educational technology company dedicated to empowering students across India. With a focus on emerging technologies like AI/ML, Data Science, Cloud computing, DevOps, Full Stack Web Development, Embedded Systems, IoT and Robotics. We've educated over 50K+ students worldwide, including those from prestigious institutions like IITs and NITs, Deemed Universities and Other engineering Colleges.Our courses are meticulously designed to equip students with practical skills for tech-driven careers, and we also provide lab setups to colleges, universities, and project solutions to companies in AI-ML, Web Development, IoT, Robotics, Cloud & DevOps domains.


In [18]:
query = "what are the courses are available on upflairs?"
query_answer = vectordb.similarity_search(query)
page_content = query_answer[0].page_content

# Split the content at "Answer:" and get the part after it
answer = page_content.split("Answer:")[1].strip()

print(answer)


Upflairs offers courses in Data Science and Machine Learning, DevOps, Full Stack Development, IoT, and System Embedding. We also provide internships to help you gain real-world experience.


In [20]:
retriever = vectordb.as_retriever(score_threshold=0.7)
query_chain = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff',retriever=retriever)
query = "how we can print welcome at upflairs 10 times?"
response = query_chain.invoke(query)
print(response['result'])

To print "Welcome at Upflairs" 10 times, you can use a simple loop in a programming language. Here's an example in Python:

```python
for i in range(10):
    print("Welcome at Upflairs")
```

This code will print "Welcome at Upflairs" 10 times.


In [21]:
def get_qa_chain(retriever):

    prompt_template = """ 
    You are a coding assistant specializing **only in Python programming**, particularly within the EdTech domain. Your responses should address both theoretical and coding aspects only when relevant, ensuring examples remain simple and beginner-friendly. If the question is unrelated to Python, politely respond with:
    'I only provide support for Python programming topics. Please ask something related to Python.'

    For each response, structure the answer by including only the relevant sections:

    1. **Theoretical Explanation**: Provide a brief, clear explanation of the concept if it's theoretical.
    2. **Code Snippet**: Include a simple code snippet if relevant. Omit this section entirely if not applicable.
    3. **Example**: Provide a clear example to illustrate the concept if helpful.

    **Important**:
    - Assume the question pertains to Python if no specific language is mentioned.
    - If another programming language is specified, respond with the predefined message.
    - For questions about Upflairs (courses, internships, or offerings), provide relevant information about Upflairs alongside any Python programming support.

    **CONTEXT**: {context}
    **My question is**: {question}
    """




    PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["context","question"]
    )

    chain = RetrievalQA.from_chain_type(llm=llm,
                                        chain_type="stuff",
                                        retriever=retriever,
                                        input_key="query",
                                        return_source_documents=True,
                                        chain_type_kwargs={"prompt": PROMPT})

    return chain


chain = get_qa_chain(retriever) 
print(chain("How we can print hello world?")['result'])


C:\Users\Ranjit\AppData\Local\Temp\ipykernel_6268\2080151199.py:40: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(chain("How we can print hello world?")['result'])


**Theoretical Explanation**:
In Python, printing text to the console is a simple operation that can be achieved using the built-in `print()` function. This function takes a string as an argument and displays it on the screen.

**Code Snippet**:
```python
print("Hello, World!")
```

**Example**:
When you run the above code, it will print "Hello, World!" to the console. This is a classic example of a "Hello, World!" program, which is often used as a beginner's introduction to programming.


In [22]:

chain = get_qa_chain(retriever) 
print(chain("How we can print hello world?")['result'])

**Theoretical Explanation**: 
In Python, printing "Hello World" is a basic operation that can be achieved using the built-in `print()` function. This function takes a string as an argument and outputs it to the console.

**Code Snippet**:
```python
print("Hello World")
```

**Example**: When you run this code, it will simply print "Hello World" to the console.


In [23]:

chain = get_qa_chain(retriever) 
print(chain("How we can print hello world in java?")['result'])

I only provide support for Python programming topics. Please ask something related to Python.


In [ ]:
chain = get_qa_chain(retriever) 
print(chain("tell me something about upflairs?")['result'])

**Theoretical Explanation**:
UpFlairs is an innovative educational technology company that offers various courses and internships to empower students across India. They focus on emerging technologies like AI/ML, Data Science, Cloud computing, DevOps, Full Stack Web Development, Embedded Systems, IoT and Robotics.

**Example**:
UpFlairs has educated over 50K+ students worldwide, including those from prestigious institutions like IITs and NITs, Deemed Universities and Other engineering Colleges. They provide lab setups to colleges, universities, and project solutions to companies in AI-ML, Web Development, IoT, Robotics, Cloud & DevOps domains. 

UpFlairs offers courses in Data Science and Machine Learning, DevOps, Full Stack Development, IoT, and System Embedding. They also provide internships to help students gain real-world experience. Additionally, they offer career guidance and support, including career counseling, resume-building workshops, and interview preparation sessions to hel